In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# download S&P 500 data pricing hystory for 5 years

sp500 = yf.download('^GSPC', period='5y')
sp500.head()

In [ ]:
# sliding mean analysis

sp500['SMA_50'] = sp500['Close'].rolling(window=50).mean()
sp500['SMA_200'] = sp500['Close'].rolling(window=200).mean()

# Plotting the closing price and the moving averages
plt.figure(figsize=(14, 7))
plt.plot(sp500['Close'], label='S&P 500 Close Price', color='blue')
plt.plot(sp500['SMA_50'], label='50-Day SMA', color='orange')
plt.plot(sp500['SMA_200'], label='200-Day SMA', color='red')
plt.title('S&P 500 Closing Price and Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# compute the daily returns
sp500['Daily Return'] = sp500['Close'].pct_change()
print(sp500['Daily Return'].describe())
# Plotting the daily returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Daily Return'], label='Daily Return', color='green')
plt.title('S&P 500 Daily Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute cumuilative returns
sp500['Cumulative Return'] = (1 + sp500['Daily Return']).cumprod()
print(sp500['Cumulative Return'].describe())
# Plotting the cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Cumulative Return'], label='Cumulative Return', color='purple')
plt.title('S&P 500 Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute rolling volatility
sp500['Rolling Volatility'] = sp500['Daily Return'].rolling(window=21).std() * (252 ** 0.5)  # Annualized volatility
print(sp500['Rolling Volatility'].describe())

# Plotting the rolling volatility and MAD
plt.figure(figsize=(14, 7))
plt.plot(sp500['Rolling Volatility'], label='Rolling Volatility (Annualized)', color='brown')
plt.title('S&P 500 Rolling Volatility (Annualized)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()  

In [ ]:
# compute MAD and quantiles
sp500['MAD'] = sp500['Close'].rolling(window=50).apply(lambda x: (x - x.mean()).abs().mean())
print(sp500['MAD'].describe())
# Plotting the Mean Absolute Deviation (MAD)
plt.figure(figsize=(14, 7))
plt.plot(sp500['MAD'], label='Mean Absolute Deviation (MAD)', color='cyan')
plt.title('S&P 500 Mean Absolute Deviation (MAD)')
plt.xlabel('Date')
plt.ylabel('MAD')
plt.legend()
plt.grid()
plt.show()      

In [ ]:
# compute 5-day, 21-day and 63-day returns
sp500['5-Day Return'] = sp500['Close'].pct_change(periods=5)
sp500['21-Day Return'] = sp500['Close'].pct_change(periods=21)
sp500['63-Day Return'] = sp500['Close'].pct_change(periods=63)

#compute 20-day and 50-day moving average
sp500['20-Day MA'] = sp500['Close'].rolling(window=20).mean()
sp500['50-Day MA'] = sp500['Close'].rolling(window=50).mean()

sp500['Signal'] = 0
sp500.loc[sp500['20-Day MA'] > sp500['50-Day MA'], 'Signal'] = 1
sp500.loc[sp500['20-Day MA'] < sp500['50-Day MA'], 'Signal'] = -1

# Plotting the 20-day and 50-day moving averages with buy/sell signals
plt.figure(figsize=(14, 7))
plt.plot(sp500['Close'], label='S&P 500 Close Price', color='blue')
plt.plot(sp500['20-Day MA'], label='20-Day MA', color='orange')
plt.plot(sp500['50-Day MA'], label='50-Day MA', color='red')
plt.scatter(sp500.index[sp500['Signal'] == 1], sp500['Close'][sp500['Signal'] == 1], label='Buy Signal', marker='^', color='green', s=100)
plt.scatter(sp500.index[sp500['Signal'] == -1], sp500['Close'][sp500['Signal'] == -1], label='Sell Signal', marker='v', color='red', s=100)
plt.title('S&P 500 Moving Averages with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()  

 Moving Average Crossover with Persistent Position

In this implementation, we interpret the trading signals as entry and exit instructions, rather than one-day actions.

Buy Signal (Signal = +1):
Enter a long position when the 20-day moving average crosses above the 50-day moving average.

Position Persistence:
After entering, remain in the long position continuously (carry it forward) until a sell signal is generated.

Sell Signal (Signal = -1):
Exit the position (return to cash exposure) when the 20-day moving average crosses back below the 50-day moving average.

Implementation:

We initialize all positions to zero.
When a buy signal occurs, the position is set to 1 (long).
We use .ffill() to carry the position forward over time.
When a sell signal occurs, the position resets to 0.

The strategy returns are computed by multiplying the lagged position by the daily returns, simulating entering positions at the close following the signal.

This approach is designed to capture sustained trends, remaining invested for the entire period between signals, rather than reacting to only the immediate day after each crossover.



In [ ]:
"""
A simple moving average crossover trading strategy and evaluate its performance 
relative to a passive buy-and-hold approach.
"""

# compute position based on the signals

sp500['Position'] = 0
sp500.loc[sp500['Signal'] == 1, 'Position'] = 1
sp500['Position'] = sp500['Position'].ffill()  # carry forward position
sp500.loc[sp500['Signal'] == -1, 'Position'] = 0

# compute cumulative returns for buy and hold strategy
sp500['Cumulative BuyHold'] = (1 + sp500['Daily Return']).cumprod()

# compute strategy returns based on the position
sp500['Strategy Return'] = sp500['Position'].shift(1) * sp500['Daily Return']
# compute cumulative strategy returns
sp500['Cumulative Strategy'] = (1 + sp500['Strategy Return']).cumprod()

# Plotting the strategy returns
plt.figure(figsize=(12, 6))
plt.plot(sp500['Cumulative Strategy'], label='Strategy Returns', color='blue')
plt.plot(sp500['Cumulative BuyHold'], label='Buy & Hold Returns', color='orange')
plt.title('Cumulative Returns: Strategy vs. Buy & Hold')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()  




### Performance metrics for the strategy

This section calculates key performance indicators (KPIs) for the strategy and compares them to a simple buy-and-hold benchmark.

####  `strategy_return`
```python
strategy_return = sp500['Cumulative Strategy'].iloc[-1] - 1
```
The total return of the strategy over the full backtest period (as a percentage). We subtract 1 to convert cumulative return into net gain.

####  `buy_hold_return`
```python
buy_hold_return = sp500['Cumulative BuyHold'].iloc[-1] - 1
```
Same as above, but for the passive buy-and-hold benchmark.

---

####  `annualized_strategy_return`
```python
annualized_strategy_return = (1 + strategy_return) ** (252 / len(sp500)) - 1
```
Annualized return of the strategy, assuming compounding.  
- `252` is the number of trading days in a year.  
- We adjust for the actual length of the dataset with `len(sp500)`.

####  `annualized_buy_hold_return`
```python
annualized_buy_hold_return = (1 + buy_hold_return) ** (252 / len(sp500)) - 1
```
Same as above, but for the buy-and-hold approach.

---

####  `annualized_volatility`
```python
annualized_volatility = sp500['Daily Return'].std() * (252 ** 0.5)
```
Measures the standard deviation (volatility) of daily returns, scaled to a full year using the square root of 252.

---

####  `sharpe_ratio`
```python
sharpe_ratio = annualized_strategy_return / annualized_volatility
```
The Sharpe ratio evaluates **risk-adjusted return**:
- Higher is better.
- It tells us how much return we earn for each unit of risk taken.
- Assumes a risk-free rate of 0%.

---

These metrics give a clearer picture of how the strategy performs not just in absolute terms, but relative to its risk and to a benchmark.


In [ ]:
# compute metrics for the strategy
strategy_return = sp500['Cumulative Strategy'].iloc[-1] - 1
buy_hold_return = sp500['Cumulative BuyHold'].iloc[-1] - 1
annualized_strategy_return = (1 + strategy_return) ** (252 / len(sp500))
annualized_buy_hold_return = (1 + buy_hold_return) ** (252 / len(sp500))
annualized_volatility = sp500['Daily Return'].std() * (252 ** 0.5)
sharpe_ratio = annualized_strategy_return / annualized_volatility   

# Print the performance metrics
print(f"Strategy Return: {strategy_return:.2%}")
print(f"Buy & Hold Return: {buy_hold_return:.2%}")
print(f"Annualized Strategy Return: {annualized_strategy_return:.2%}")
print(f"Annualized Buy & Hold Return: {annualized_buy_hold_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}") 

### Strategy Performance Discussion

As the cumulative returns plot clearly shows, this simple moving average crossover strategy underperforms the buy-and-hold benchmark—by quite a margin.

A few likely reasons for that:

- The strategy is slow to react: it often enters late and exits early, missing out on the core of an uptrend.
- It tends to get whipsawed in choppy markets—exiting on minor pullbacks and re-entering too late.
- And of course, over this period, the S&P 500 has been on a relatively strong and consistent uptrend—so staying invested without trying to time the market was actually the better choice.

That said, I think this still serves as a good base. It highlights how important it is to:

- Evaluate strategies on different market regimes (not just bull runs),
- Consider out-of-sample performance seriously,
- And maybe think about adding a few improvements—like a volatility filter, or replacing moving averages with something more responsive (RSI, MACD, or even basic price momentum).

Even if this version doesn’t “beat the market,” it sets up a clean, testable framework for exploring new ideas.


In [ ]:
# adding transaction costs
transaction_cost = 0.001  # 0.1% transaction cost
sp500['Transaction Cost'] = transaction_cost * sp500['Position'].diff().abs()
sp500['Strategy Return With Costs'] = sp500['Strategy Return'] - sp500['Transaction Cost']
sp500['Cumulative Strategy With Costs'] = (1 + sp500['Strategy Return With Costs']).cumprod()   

# Plotting the strategy returns with transaction costs
plt.figure(figsize=(12, 6))
plt.plot(sp500['Cumulative Strategy With Costs'], label='Strategy Returns with Costs', color='purple')
plt.plot(sp500['Cumulative Strategy'], label='Strategy Returns without Costs', color='blue')
plt.plot(sp500['Cumulative BuyHold'], label='Buy & Hold Returns', color='orange')
plt.title('Cumulative Returns: Strategy vs. Buy & Hold (with Costs)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()    